In [1]:
from edgedroid.models.timings import *

runs_per_model = 30
task_steps = 100

models = {
    "empirical": {
        "low": EmpiricalExecutionTimeModel.from_default_data(neuroticism=0.0),
        "high": EmpiricalExecutionTimeModel.from_default_data(neuroticism=1.0),
    },
    "theoretical": {
        "low": TheoreticalExecutionTimeModel.from_default_data(neuroticism=0.0),
        "high": TheoreticalExecutionTimeModel.from_default_data(neuroticism=1.0),
    },
    "constant": {"n/a": ConstantExecutionTimeModel.from_default_data()},
    "naive": {"n/a": NaiveExecutionTimeModel.from_default_data()},
    "fitted-naive": {"n/a": FittedNaiveExecutionTimeModel.from_default_data()},
}

In [2]:
import numpy as np

ttfs = np.linspace(0, 5, 25)
ttfs

array([0.        , 0.20833333, 0.41666667, 0.625     , 0.83333333,
       1.04166667, 1.25      , 1.45833333, 1.66666667, 1.875     ,
       2.08333333, 2.29166667, 2.5       , 2.70833333, 2.91666667,
       3.125     , 3.33333333, 3.54166667, 3.75      , 3.95833333,
       4.16666667, 4.375     , 4.58333333, 4.79166667, 5.        ])

In [3]:
import pandas as pd
import itertools as it

combs = deque()
for model_name, variants in models.items():
    for neuro, model in variants.items():
        for ttf, run in it.product(ttfs, range(1, runs_per_model + 1)):
            combs.append((model, model_name, neuro, ttf, run))

In [4]:
from tqdm.notebook import tqdm_notebook as tqdm

curr_idx = 0
tqdm_iter = tqdm(combs)

rows = deque()

for model, model_name, neuro, ttf, repetition in tqdm_iter:
    tqdm_iter.set_description(
        desc=f"Model {model_name}, neuro {neuro}, TTF {ttf:0.02f}s, repetition {repetition}"
    )
    model.reset()
    cum_duration = 0.0

    for step in range(1, task_steps + 1):
        exec_time = model.advance(ttf).get_execution_time()
        duration = exec_time + ttf
        cum_duration += duration

        rows.append(
            {
                "model": model_name,
                "neuroticism": neuro,
                "repetition": repetition,
                "step": step,
                "exec_time": exec_time,
                "ttf": ttf,
                "duration": duration,
                "cumulative_duration": cum_duration,
            }
        )

        # results.loc[curr_idx, ["model", "neuroticism", "repetition", "step", "exec_time", "ttf", "duration", "cumulative_duration"]] = \
        #     [model_name, neuro, repetition, step, exec_time, ttf, duration, cum_duration]


        # model_name
        # results.at[curr_idx, "neuroticism"] = neuro
        # results.at[curr_idx, "repetition"] = repetition
        # results.at[curr_idx, "step"] = step
        # results.at[curr_idx, "exec_time"] = exec_time
        # results.at[curr_idx, "ttf"] = ttf
        # results.at[curr_idx, "duration"] = duration
        # results.at[curr_idx, "cumulative_duration"] = cum_duration

        curr_idx += 1

results = pd.DataFrame(rows)
results["model"] = results["model"].astype(pd.CategoricalDtype(models.keys(), ordered=False))
results["neuroticism"] = results["neuroticism"].astype(pd.CategoricalDtype(["n/a", "low", "high"], ordered=True))
results["repetition"] = results["repetition"].astype(int)
results["step"] = results["step"].astype(int)
results["exec_time"] = results["exec_time"].astype(float)
results["ttf"] = results["ttf"].astype(float)
results["duration"] = results["duration"].astype(float)
results["cumulative_duration"] = results["cumulative_duration"].astype(float)

results.to_parquet("./timing_models_scaling_ttf.gzip", compression="gzip", partition_cols=["model"])

results

  0%|          | 0/5250 [00:00<?, ?it/s]

,model,neuroticism,repetition,step,exec_time,ttf,duration,cumulative_duration
0,empirical,low,1,1,2.689960,0.0,2.689960,2.689960
1,empirical,low,1,2,5.041830,0.0,5.041830,7.731790
2,empirical,low,1,3,6.028927,0.0,6.028927,13.760717
3,empirical,low,1,4,8.054360,0.0,8.054360,21.815077
4,empirical,low,1,5,4.063269,0.0,4.063269,25.878346
...,...,...,...,...,...,...,...,...
524995,fitted-naive,n/a,30,96,3.033511,5.0,8.033511,1020.405085
524996,fitted-naive,n/a,30,97,4.314555,5.0,9.314555,1029.719640
524997,fitted-naive,n/a,30,98,4.829962,5.0,9.829962,1039.549602
524998,fitted-naive,n/a,30,99,3.470785,5.0,8.470785,1048.020388
